In [1]:
import typesense
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
client = typesense.Client({
    'nodes':[{
        'host':'4pd87kfnbcvu1xs2p-1.a1.typesense.net',
        'port':'443',
        'protocol':'https'
    }],
    'api_key':os.getenv('typesense_API'),
    'connection_timeout_seconds':2
})

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI


In [4]:
loader = TextLoader('../data/text_file/text_split.txt')
Document = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(Document)

In [5]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_24783/3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_24783/3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/ayush/Documents/AI/Projects/Rag_Application/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.htm

In [6]:
vectorDB = Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "4pd87kfnbcvu1xs2p-1.a1.typesense.net",  
        "port": "443",  
        "protocol": "https",  
        "typesense_api_key":os.getenv('typesense_API'),
        "typesense_collection_name": "lang-chain"
    }
)

In [8]:
query = "explain Quoting"
found_docs = vectorDB.similarity_search(query)
print(found_docs[0].page_content)

--- Chunk 36 ---
supported in bash. It uses backquotes instead of the dollar sign and parentheses.
[me@linuxbox ~]$ ls -l `which cp`
-rwxr-xr-x 1 root root 71516 2007-12-05 08:58 /bin/cp
Quoting
Now that we've seen how many ways the shell can perform expansions, it's time to learn
how we can control it. Take for example the following:
[me@linuxbox ~]$ echo this is a     test
this is a test
or this one:
75

--- Chunk 37 ---
E. & O.E.
page 1 of 1
*Keep this invoice and 
manufacturer box for 
warranty purposes.
Ship To
Santosh Kumar 
Plot no74 sawastik enclev, Janpath 
road fulsunga.
Rudrapur 263153 Uttarakhand 
Phone: xxxxxxxxxx 
Bill To
Santosh Kumar 
Plot no74 sawastik enclev, Janpath 
road fulsunga.
Rudrapur 263153 Uttarakhand 
Phone: xxxxxxxxxx 
Order ID: OD126318642060163000
 21-10-2022
Order Date:
 22-10-2022
Invoice Date:
 AAFCI2112P
PAN:
 U51900DL2019PTC347263
CIN:
Invoice Number # FAFO7V2300034840


In [9]:
retriever = vectorDB.as_retriever()

In [16]:
context = retriever.invoke(query)[0].page_content

In [20]:
llm = ChatGoogleGenerativeAI(
    api_key=os.getenv('GEMINI_API'),
    model='gemini-2.5-flash-lite',
    max_output_tokens=1024,
    temperature=0.5
)
query1 = "explain Quoting in 100 words"
prompt=f"""Use the following context to answer the question concisely.
            Context:
            {context}

            Question: {query1}

            Answer:"""
response = llm.invoke([prompt])
response.content

'Quoting in shell scripting allows you to control how the shell interprets special characters and commands. It prevents unintended expansions or execution of commands embedded within strings. Common quoting mechanisms include single quotes (`\'`) which treat everything literally, and double quotes (`"`) which allow for variable expansion and command substitution but prevent word splitting and globbing. Backticks (`` ` ``) are also used for command substitution, executing a command and replacing it with its output. Mastering quoting is essential for writing robust and predictable shell scripts.'